In [ ]:
def recursive_find_adjs(root, sentence):
    children = [w for w in sentence.words if w.head == root.id]
    if not children:
        pass 
    filtered_child = [w for w in children if (w.deprel == "conj" or w.deprel == "appos" 
                                              or w.deprel == "nmod" #w.deprel == or w.deprel == "compound" or w.deprel == "nsubj"
                                              or w.deprel == "amod") and (w.pos == "ADJ"or w.pos == 'NOUN')] 
    results = [w for w in filtered_child if not any(sub.head == w.id and sub.upos == "NOUN" for sub in sentence.words)]
    for w in children:
        results += recursive_find_adjs(w, sentence)
    return results

In [ ]:
def char_attributes(doc):
    sentences = []
    names = []
    attributes = []
    for ix, sent in enumerate(doc.sentences):

      nouns = [w for w in sent.words if (w.pos == "PROPN" or w.pos == "PRON")]
      
      name_list = list(char_table_all.loc[char_table_all['sentence']==ix]['name'])
      for noun in nouns:
        if noun.text in name_list:
            adj0 = sent.words[noun.head-1] #adjective directly related
            adjs = [adj0] + recursive_find_adjs(adj0, sent) if adj0.pos == "ADJ" or adj0.pos == "NOUN" else []
            '''deps = [w for w in sent.words if (w.head == noun.id and (w.upos == 'NOUN' or w.upos == 'ADJ')) ]
            for i in range(len(deps)):
              adj0 = deps[i]
              print(adj0.text)
              print('\n')
              adjs = [adj0] + recursive_find_adjs(adj0, sent) if adj0.pos == "ADJ" or adj0.pos == "NOUN" else [] '''
            mod_adjs = [w for w in sent.words if w.head == noun.id and (w.pos == "ADJ" or w.pos == 'NOUN')]
              # This should only be one element because conjunctions are hierarchical
            if mod_adjs:
              mod_adj = mod_adjs[0]
              adjs.extend([mod_adj] + recursive_find_adjs(mod_adj, sent))
            if adjs:
              unique_adjs = []
              unique_ids = set()
              for adj in adjs:
                  if adj.id not in unique_ids:
                      unique_adjs.append(adj)
                      unique_ids.add(adj.id)
              sentences.append(ix)
              names.append(noun.text)
              attributes.append(" ".join([adj.text for adj in unique_adjs]))
    char_attributes = pd.DataFrame()
    char_attributes['Sentence'] = sentences
    char_attributes['Character Names'] = names
    char_attributes['Character Attributes'] = attributes
    char_attributes['Total Attributes'] = char_attributes.groupby(['Sentence','Character Names'])['Character Attributes'].transform(lambda x: ' '.join(x))
    char_attributes= char_attributes[['Sentence', 'Character Names','Total Attributes']]
    return (char_attributes.drop_duplicates().reset_index())